In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
path = "/content/gdrive/My Drive/Semester 7/NLP/Assignment3/"
with open(path + "train.txt", 'r') as ftrain:
    train_text = ftrain.read()
with open(path + "test.txt", 'r') as ftest:
    test_text = ftest.read()

In [0]:
import nltk
# nltk.download('stopwords')
from nltk.corpus import stopwords
import re

In [0]:
help(stopwords)

In [5]:
stopwords_en = stopwords.words('english')
exclude = ['very', 'not', 'never', 'no', 'ever', 'nothing', 'really', 'extremely']
for i in exclude:
    if i not in stopwords_en:
        print(i, end=' ')

never ever nothing really extremely 

In [0]:
emojis = ['😂', '❤', '♥', '😍', '😭', '😘', '😊', '👌', '💕', '👏', '😁', '☺', '♡', '👍', '😩', '🙏', '✌', '😏', '😉', '🙌',
     '🙈', '💪', '😄', '😒', '💃', '💖', '😃', '😔', '😱', '🎉', '😜', '☯', '🌸', '💜', '💙', '✨', '😳', '💗', '★',
     '☀', '😡', '😎', '😢', '💋', '😋', '🙊', '😴', '🎶', '💞', '😌']
emoji_dict = {emojis[i]: i for i in range(len(emojis))}
# 50 most frequently used emojis from https://www.kaggle.com/thomasseleck/emoji-sentiment-data

In [0]:
train_id = []
train_data = []
train_hin, train_eng, train_o, train_labels = [], [], [], []
# emoji_train = []
for sample in train_text.split('\n\n'):
    
    lines_sample = sample.split('\n')
    # print(lines_sample[0])
    try:
        train_labels.append(lines_sample[0].split()[2])
        train_id.append(lines_sample[0].split()[1])
    except IndexError:
        del train_id[-1]
        continue
    temp, temp_eng, temp_hin, temp_o = [], [], [], []
    
    
    for line in lines_sample[1:]:
        t = line.split('\t')

        if t[1] != 'O':
            t[0]=re.sub('[\W_]+', '', t[0])
        if t[1] == 'Eng' and t[0] in stopwords_en and t[0] not in exclude:
            continue
        if 'http' in t[0]:
            continue
        temp.append(t[0])
        if t[1] == 'Eng':
            temp_eng.append(t[0])
        elif t[1] == 'Hin':
            temp_hin.append(t[0])
        elif t[1] == 'O':
            temp_o.append(t[0])
    if temp == []:
        continue
    train_data.append(temp)
    train_eng.append(temp_eng)
    train_hin.append(temp_hin)
    train_o.append(temp_o)


In [8]:
print(len(train_text.split('\n\n')))

15132


In [9]:
len(train_data)

15131

In [0]:
test_id = []
test_data = []
test_hin, test_eng, test_o, test_labels = [], [], [], []

for sample in test_text.split('\n\n'):
    
    lines_sample = sample.split('\n')
    # print(lines_sample[0])
    try:
        test_labels.append(lines_sample[0].split()[2])
        test_id.append(lines_sample[0].split()[1])
    except IndexError:
        del test_id[-1]
        continue
    temp, temp_eng, temp_hin, temp_o = [], [], [], []
    
    
    for line in lines_sample[1:]:
        t = line.split('\t')
        t[0] = t[0].lower()
        if t[1] != 'O':
            t[0]=re.sub('[\W_]+', '', t[0])
        if t[1] == 'Eng' and t[0] in stopwords_en and t[0] not in exclude:
            continue
        if 'http' in t[0]:
            continue
        temp.append(t[0])
        if t[1] == 'Eng':
            temp_eng.append(t[0])
        elif t[1] == 'Hin':
            temp_hin.append(t[0])
        elif t[1] == 'O':
            temp_o.append(t[0])
    if temp == []:
        continue
    test_data.append(temp)
    test_eng.append(temp_eng)
    test_hin.append(temp_hin)
    test_o.append(temp_o)

In [0]:
train_tweets = [' '.join(i) for i in train_data]
test_tweets = [' '.join(i) for i in test_data]

In [0]:
train_tweets_dict = {}
test_tweets_dict = {}

In [0]:
train_tweets_dict['eng'] = [' '.join(i) for i in train_eng]
test_tweets_dict['eng'] = [' '.join(i) for i in test_eng]

In [0]:
train_tweets_dict['hin'] = [' '.join(i) for i in train_hin]
test_tweets_dict['hin'] = [' '.join(i) for i in test_hin]

In [0]:
train_tweets_dict['o'] = [' '.join(i) for i in train_o]
test_tweets_dict['o'] = [' '.join(i) for i in test_o]

In [122]:
np.unique(train_labels)

array(['negative', 'neutral', 'positive'], dtype='<U8')

In [22]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from keras.preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split
import pandas as pd
import random
import numpy as np
from keras.preprocessing import sequence
from keras.utils import np_utils

from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Lambda
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM, SimpleRNN, GRU
from keras.preprocessing.text import Tokenizer
from keras import optimizers

Using TensorFlow backend.


In [0]:
label_values = {'negative':0, 'neutral':1, 'positive':2}

In [0]:
y_train = np.array([label_values[i] for i in train_labels])
y_test = np.array([label_values[i] for i in test_labels])

In [0]:
max_features = 20000
tokenizer1 = Tokenizer(num_words=max_features)
tokenizer1.fit_on_texts(train_tweets)

In [28]:
max_len = 250
num_classes = 3

sequences_train = tokenizer1.texts_to_sequences(train_tweets)
sequences_test = tokenizer1.texts_to_sequences(test_tweets)

X_train = sequence.pad_sequences(sequences_train, maxlen=max_len)
X_test = sequence.pad_sequences(sequences_test, maxlen=max_len)

Y_train = np_utils.to_categorical(y_train, num_classes)
Y_test = np_utils.to_categorical(y_test, num_classes)

print('X_train shape:', X_train.shape)
print('X_test shape:', X_test.shape)

X_train shape: (15131, 250)
X_test shape: (1869, 250)


In [0]:
input_dim = X_train.shape[1]

In [0]:
input_dim = X_train.shape[1]
model1 = Sequential()

model1.add(Dense(input_dim))
model1.add(Dropout(0.2))
model1.add(Activation('relu'))
model1.add(Dense(200))
model1.add(Dropout(0.2))
model1.add(Activation('tanh'))
model1.add(Dense(100))
model1.add(Dropout(0.2))
model1.add(Activation('sigmoid'))
model1.add(Dense(3))
model1.add(Activation('softmax'))

adam = optimizers.Adam(lr=0.01, decay=1e-6)

model1.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])

In [34]:
model1.fit(X_train, Y_train, batch_size = 256, epochs=1)

Epoch 1/1
15131/15131 [==============================] - 1s 70us/step - loss: 0.6535 - acc: 0.6507


In [35]:
preds = model1.predict_classes(X_test, verbose=0)
np.sum(preds==y_test)/len(y_test)

0.4002140181915463

In [36]:
model1.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_5 (Dense)              (None, 250)               62750     
_________________________________________________________________
dropout_4 (Dropout)          (None, 250)               0         
_________________________________________________________________
activation_5 (Activation)    (None, 250)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 200)               50200     
_________________________________________________________________
dropout_5 (Dropout)          (None, 200)               0         
_________________________________________________________________
activation_6 (Activation)    (None, 200)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 100)              

In [0]:
tokenizers = {}
for v in ['eng', 'hin', 'o']:
    tokenizers[v] = Tokenizer(num_words=max_features)
    tokenizers[v].fit_on_texts(train_tweets_dict[v])

In [38]:
sequences_train_dict, sequences_test_dict = {}, {}
X_train_dict, X_test_dict = {}, {}
for v in ['eng', 'hin', 'o']:
    sequences_train_dict[v] = tokenizers[v].texts_to_sequences(train_tweets)
    sequences_test_dict[v] = tokenizers[v].texts_to_sequences(test_tweets)

    X_train_dict[v] = sequence.pad_sequences(sequences_train_dict[v], maxlen=max_len)
    X_test_dict[v] = sequence.pad_sequences(sequences_test_dict[v], maxlen=max_len)

# Y_train = np_utils.to_categorical(y_train, num_classes)
# Y_test = np_utils.to_categorical(y_test, num_classes)

X_train2 = np.hstack(tuple([X_train] + [X_train_dict[v] for v in ['eng', 'hin', 'o']]))
X_test2 = np.hstack(tuple([X_test] + [X_test_dict[v] for v in ['eng', 'hin', 'o']]))

print('X_train2 shape:', X_train2.shape)
print('X_test2 shape:', X_test2.shape)

X_train2 shape: (15131, 1000)
X_test2 shape: (1869, 1000)


In [0]:
# sum(np.sum(emoji_train, axis=0) > 0)

In [0]:
input_dim2 = X_train2.shape[1]
model2 = Sequential()

model2.add(Dense(input_dim))
model2.add(Dropout(0.2))
model2.add(Activation('relu'))
model2.add(Dense(200))
model2.add(Dropout(0.2))
model2.add(Activation('tanh'))
model2.add(Dense(100))
model2.add(Dropout(0.2))
model2.add(Activation('sigmoid'))
model2.add(Dense(3))
model2.add(Activation('softmax'))

adam = optimizers.Adam(lr=0.001, decay=1e-6)

model2.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])

In [42]:
model2.fit(X_train2, Y_train, batch_size = 256, epochs=1)

Epoch 1/1
15131/15131 [==============================] - 2s 113us/step - loss: 0.6664 - acc: 0.6436


In [43]:
preds2 = model2.predict_classes(X_test2, verbose=0)
np.sum(preds2==y_test)/len(y_test)

0.4071696094168004

In [44]:
preds2[:10]

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [45]:
X_train3 = X_train.copy()
X_test3 = X_test.copy()

emoji_train = np.zeros((X_train.shape[0], 50))
emoji_test = np.zeros((X_test.shape[0], 50))

i = 0
for sample in train_text.split('\n\n'):
    
    lines_sample = sample.split('\n')
    try:
        tmp = lines_sample[0].split()[2]
        tmp = lines_sample[0].split()[1]
    except IndexError:
        continue    
    
    for line in lines_sample[1:]:
        t = line.split('\t')
        for ch in t[0]:
            if ch in emojis:
                emoji_train[i][emoji_dict[ch]] += 1
    i += 1

i = 0
for sample in test_text.split('\n\n'):
    
    lines_sample = sample.split('\n')
    try:
        tmp = lines_sample[0].split()[2]
        tmp = lines_sample[0].split()[1]
    except IndexError:
        continue

    for line in lines_sample[1:]:
        t = line.split('\t')
        for ch in t[0]:
            if ch in emojis:
                emoji_test[i][emoji_dict[ch]] += 1
    i += 1

X_train3 = np.hstack((X_train, emoji_train))
X_test3 = np.hstack((X_test, emoji_test))

print('X_train3 shape:', X_train3.shape)
print('X_test3 shape:', X_test3.shape)

X_train3 shape: (15131, 300)
X_test3 shape: (1869, 300)


In [46]:
sum(np.sum(emoji_train, axis=1)>0)

2112

In [0]:
input_dim3 = X_train3.shape[1]
model3 = Sequential()

model3.add(Dense(input_dim))
model3.add(Dropout(0.2))
model3.add(Activation('relu'))
model3.add(Dense(200))
model3.add(Dropout(0.2))
model3.add(Activation('tanh'))
model3.add(Dense(100))
model3.add(Dropout(0.2))
model3.add(Activation('sigmoid'))
model3.add(Dense(3))
model3.add(Activation('softmax'))

adam = optimizers.Adam(lr=0.001, decay=1e-6)

model3.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])

In [60]:
model3.fit(X_train3, Y_train, batch_size = 512, epochs=4)

Epoch 1/4
15131/15131 [==============================] - 2s 115us/step - loss: 0.7066 - acc: 0.6207
Epoch 2/4
15131/15131 [==============================] - 1s 38us/step - loss: 0.6396 - acc: 0.6603
Epoch 3/4
15131/15131 [==============================] - 1s 38us/step - loss: 0.6359 - acc: 0.6626
Epoch 4/4
15131/15131 [==============================] - 1s 38us/step - loss: 0.6343 - acc: 0.6631


In [61]:
preds3 = model3.predict_classes(X_test3, verbose=0)
np.sum(preds3==y_test)/len(y_test)

0.32691278758694486

In [62]:
sum(sum(emoji_train))

5102.0

In [149]:
X_train.shape

(15131, 250)

In [0]:
from sklearn.tree import DecisionTreeClassifier

In [65]:
dtree1 = DecisionTreeClassifier()
dtree1.fit(X_train, Y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

In [66]:
predsd1 = dtree1.predict(X_test)
predsd1 = np.argmax(predsd1, axis=1)
np.sum(predsd1==y_test)/len(y_test)

0.3911182450508293

In [67]:
dtree2 = DecisionTreeClassifier()
dtree2.fit(X_train2, Y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

In [68]:
predsd2 = dtree2.predict(X_test2)
predsd2 = np.argmax(predsd2, axis=1)
np.sum(predsd2==y_test)/len(y_test)

0.38095238095238093

In [69]:
dtree3 = DecisionTreeClassifier()
dtree3.fit(X_train3, Y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

In [70]:
predsd3 = dtree3.predict(X_test3)
predsd3 = np.argmax(predsd3, axis=1)
np.sum(predsd3==y_test)/len(y_test)

0.39058319957196364

In [78]:
a = X_test3[(np.sum(emoji_test, axis=1)>0)]
p = y_test[(np.sum(emoji_test, axis=1)>0)]
predsd4 = dtree3.predict(a)
predsd4 = np.argmax(predsd4, axis=1)
np.sum(predsd4==p)/len(p)

0.4262295081967213

In [0]:
from keras.layers.normalization import BatchNormalization
from keras.layers import SpatialDropout1D
from keras.models import Model
from keras.layers import Input,Flatten, Dense, Embedding, RNN, Conv1D, BatchNormalization, MaxPooling1D, Activation, Dropout, concatenate, Lambda
from keras import optimizers
from keras.layers.convolutional import Convolution1D
from keras import backend as K

In [106]:
nb_filter = 300
filter_length = 3
hidden_dims = 300 # 250
nb_epoch = 2


cmodel1 = Sequential()
cmodel1.add(Embedding(max_features, 300))
cmodel1.add(SpatialDropout1D(0.2))
# we add a Convolution1D, which will learn nb_filter
# word group filters of size filter_length:
cmodel1.add(Convolution1D(nb_filter=nb_filter,
                       filter_length=filter_length,
                       border_mode='valid',
                       activation='tanh',
                       subsample_length=1))

#cmodel1.add(BatchNormalization())
from keras import optimizers
def max_1d(X):
    return K.max(X, axis=1)

cmodel1.add(Lambda(max_1d, output_shape=(nb_filter,)))
cmodel1.add(Dense(hidden_dims))
cmodel1.add(Dropout(0.2))
cmodel1.add(Activation('relu'))
cmodel1.add(Dense(num_classes))
cmodel1.add(Activation('sigmoid'))
adam = optimizers.Adam(lr=0.001, decay=1e-6)
cmodel1.compile(loss='binary_crossentropy',
             optimizer=adam,
             metrics=['accuracy'])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(activation="tanh", filters=300, kernel_size=3, strides=1, padding="valid")`
  app.launch_new_instance()


In [107]:
cmodel1.fit(X_train3, Y_train, epochs = 1)

Epoch 1/1
15131/15131 [==============================] - 237s 16ms/step - loss: 0.5342 - acc: 0.7211


In [109]:
predsc1 = cmodel1.predict_classes(X_test3, verbose=0)
np.sum(predsc1==y_test)/len(y_test)

0.565008025682183

In [167]:
prf(y_test, predsc1, average='micro')

(0.565008025682183, 0.565008025682183, 0.565008025682183, None)

In [168]:
prf(y_test, predsc1, average='macro')

(0.5659138903963613, 0.5840501910447199, 0.5662953882918141, None)

In [169]:
prf(y_test, predsc1, average='weighted')

(0.5675574871168725, 0.565008025682183, 0.5569762553083624, None)

In [108]:
cmodel1.summary()

Model: "sequential_20"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_12 (Embedding)     (None, None, 300)         6000000   
_________________________________________________________________
spatial_dropout1d_11 (Spatia (None, None, 300)         0         
_________________________________________________________________
conv1d_9 (Conv1D)            (None, None, 300)         270300    
_________________________________________________________________
lambda_9 (Lambda)            (None, 300)               0         
_________________________________________________________________
dense_46 (Dense)             (None, 300)               90300     
_________________________________________________________________
dropout_32 (Dropout)         (None, 300)               0         
_________________________________________________________________
activation_46 (Activation)   (None, 300)             

In [145]:
nb_filter = 300
filter_length = 3
hidden_dims = 300 # 250
nb_epoch = 2


cmodel1a = Sequential()
cmodel1a.add(Embedding(max_features, 300))
cmodel1a.add(SpatialDropout1D(0.2))
# we add a Convolution1D, which will learn nb_filter
# word group filters of size filter_length:
cmodel1a.add(Convolution1D(nb_filter=nb_filter,
                       filter_length=filter_length,
                       border_mode='valid',
                       activation='tanh',
                       subsample_length=1))

cmodel1a.add(Lambda(max_1d, output_shape=(nb_filter,)))
cmodel1a.add(Dense(hidden_dims))
cmodel1a.add(Dropout(0.2))
cmodel1a.add(Activation('relu'))
cmodel1a.add(Dense(num_classes))
cmodel1a.add(Activation('sigmoid'))
adam = optimizers.Adam(lr=0.001, decay=1e-6)
cmodel1a.compile(loss='binary_crossentropy',
             optimizer=adam,
             metrics=['accuracy'])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(activation="tanh", filters=300, kernel_size=3, strides=1, padding="valid")`
  app.launch_new_instance()


In [146]:
cmodel1a.fit(X_train3, Y_train, epochs = 2, batch_size=256)

Epoch 1/2
15131/15131 [==============================] - 173s 11ms/step - loss: 0.5879 - acc: 0.6856
Epoch 2/2
15131/15131 [==============================] - 168s 11ms/step - loss: 0.4583 - acc: 0.7792


In [147]:
predsc1a = cmodel1a.predict_classes(X_test3, verbose=0)
np.sum(predsc1a==y_test)/len(y_test)

0.5644729802033173

In [0]:
from sklearn.metrics import precision_recall_fscore_support as prf

In [163]:
prf(y_test, predsc1a, average='micro')

(0.5644729802033173, 0.5644729802033173, 0.5644729802033173, None)

In [164]:
prf(y_test, predsc1a, average='macro')

(0.5708475748562786, 0.563563693418115, 0.5666094135651139, None)

In [166]:
prf(y_test, predsc1a, average='weighted')

(0.5663970904202026, 0.5644729802033173, 0.5648397339912556, None)

In [132]:
cmodel2 = Sequential()
cmodel2.add(Embedding(max_features, 500))
cmodel2.add(SpatialDropout1D(0.2))
# we add a Convolution1D, which will learn nb_filter
# word group filters of size filter_length:
cmodel2.add(Convolution1D(nb_filter=nb_filter,
                       filter_length=filter_length,
                       border_mode='valid',
                       activation='tanh',
                       subsample_length=1))

cmodel2.add(Lambda(max_1d, output_shape=(nb_filter,)))
cmodel2.add(Dense(hidden_dims))
cmodel2.add(Dropout(0.2))
cmodel2.add(Activation('relu'))
cmodel2.add(Dense(num_classes))
cmodel2.add(Activation('sigmoid'))
adam = optimizers.Adam(lr=0.001, decay=1e-6)
cmodel2.compile(loss='binary_crossentropy',
             optimizer=adam,
             metrics=['accuracy'])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(activation="tanh", filters=300, kernel_size=3, strides=1, padding="valid")`
  # Remove the CWD from sys.path while we load stuff.


In [133]:
cmodel2.fit(X_train2, Y_train, epochs = 1)

Epoch 1/1
15131/15131 [==============================] - 986s 65ms/step - loss: 0.5525 - acc: 0.7080


In [134]:
predsc2 = cmodel2.predict_classes(X_test3, verbose=0)
np.sum(predsc2==y_test)/len(y_test)

0.46441947565543074

In [117]:
cmodel2.summary()

Model: "sequential_22"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_14 (Embedding)     (None, None, 500)         10000000  
_________________________________________________________________
spatial_dropout1d_13 (Spatia (None, None, 500)         0         
_________________________________________________________________
conv1d_11 (Conv1D)           (None, None, 300)         450300    
_________________________________________________________________
lambda_11 (Lambda)           (None, 300)               0         
_________________________________________________________________
dense_50 (Dense)             (None, 300)               90300     
_________________________________________________________________
dropout_34 (Dropout)         (None, 300)               0         
_________________________________________________________________
activation_50 (Activation)   (None, 300)             

In [0]:
!pip install emoji
import emoji

In [0]:
new_train_data = []

for sample in train_text.split('\n\n'):
    
    lines_sample = sample.split('\n')
    try:
        tmp = (lines_sample[0].split()[2])
        tmp = (lines_sample[0].split()[1])
    except IndexError:
        continue
    temp = []
    
    for line in lines_sample[1:]:
        t = line.split('\t')

        if t[1] != 'O':
            t[0]=re.sub('[\W_]+', '', t[0])
            new = ''
            for ch in t[0]:
                if ch in emojis:
                    new += ' ' + emoji.demojize(ch) + ' '
        if t[1] == 'Eng' and t[0] in stopwords_en and t[0] not in exclude:
            continue
        if 'http' in t[0]:
            continue
        temp.append(t[0])
    if temp == []:
        continue
    new_train_data.append(temp)


In [0]:
new_test_data = []
# emoji_test = []
for sample in test_text.split('\n\n'):
    
    lines_sample = sample.split('\n')
    try:
        tmp = (lines_sample[0].split()[2])
        tmp = (lines_sample[0].split()[1])
    except IndexError:
        continue
    temp = []
    
    for line in lines_sample[1:]:
        t = line.split('\t')

        if t[1] != 'O':
            t[0]=re.sub('[\W_]+', '', t[0])
            new = ''
            for ch in t[0]:
                if ch in emojis:
                    new += ' ' + emoji.demojize(ch) + ' '
        if t[1] == 'Eng' and t[0] in stopwords_en and t[0] not in exclude:
            continue
        if 'http' in t[0]:
            continue
        temp.append(t[0])
    if temp == []:
        continue
    new_test_data.append(temp)

In [0]:
new_train_tweets = [' '.join(i) for i in new_train_data]
new_test_tweets = [' '.join(i) for i in new_test_data]

In [0]:
max_features = 20000
tokenizer2 = Tokenizer(num_words=max_features)
tokenizer2.fit_on_texts(new_train_tweets)

In [126]:
max_len = 250
num_classes = 3

new_sequences_train = tokenizer2.texts_to_sequences(new_train_tweets)
new_sequences_test = tokenizer2.texts_to_sequences(new_test_tweets)

X_train4 = sequence.pad_sequences(new_sequences_train, maxlen=max_len)
X_test4 = sequence.pad_sequences(new_sequences_test, maxlen=max_len)

# Y_train = np_utils.to_categorical(y_train, num_classes)
# Y_test = np_utils.to_categorical(y_test, num_classes)

print('X_train4 shape:', X_train4.shape)
print('X_test4 shape:', X_test4.shape)

X_train4 shape: (15131, 250)
X_test4 shape: (1869, 250)


In [127]:
cmodel3 = Sequential()
cmodel3.add(Embedding(max_features, 300))
cmodel3.add(SpatialDropout1D(0.2))
# we add a Convolution1D, which will learn nb_filter
# word group filters of size filter_length:
cmodel3.add(Convolution1D(nb_filter=nb_filter,
                       filter_length=filter_length,
                       border_mode='valid',
                       activation='tanh',
                       subsample_length=1))

cmodel3.add(Lambda(max_1d, output_shape=(nb_filter,)))
cmodel3.add(Dense(hidden_dims))
cmodel3.add(Dropout(0.2))
cmodel3.add(Activation('relu'))
cmodel3.add(Dense(num_classes))
cmodel3.add(Activation('sigmoid'))
adam = optimizers.Adam(lr=0.001, decay=1e-6)
cmodel3.compile(loss='binary_crossentropy',
             optimizer=adam,
             metrics=['accuracy'])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(activation="tanh", filters=300, kernel_size=3, strides=1, padding="valid")`
  # Remove the CWD from sys.path while we load stuff.


In [129]:
cmodel3.fit(X_train4, Y_train, epochs = 1)

Epoch 1/1
15131/15131 [==============================] - 216s 14ms/step - loss: 0.5303 - acc: 0.7264


In [130]:
predsc3 = cmodel3.predict_classes(X_test4, verbose=0)
np.sum(predsc3==y_test)/len(y_test)

0.5521669341894061

In [131]:
cmodel3.summary()

Model: "sequential_23"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_15 (Embedding)     (None, None, 300)         6000000   
_________________________________________________________________
spatial_dropout1d_14 (Spatia (None, None, 300)         0         
_________________________________________________________________
conv1d_12 (Conv1D)           (None, None, 300)         270300    
_________________________________________________________________
lambda_12 (Lambda)           (None, 300)               0         
_________________________________________________________________
dense_52 (Dense)             (None, 300)               90300     
_________________________________________________________________
dropout_35 (Dropout)         (None, 300)               0         
_________________________________________________________________
activation_52 (Activation)   (None, 300)             

In [126]:
print(train_text[:100])

meta	3	negative
@	O
AdilNisarButt	Hin
pakistan	Hin
ka	Hin
ghra	Hin
tauq	Hin
he	Eng
Pakistan	Eng
Isra


In [57]:
a = np.array([[1,1,1,1], [2,2,2,2]])
b = np.array([[3,3],[4,4]])
print(a)
print(b)
print(np.hstack((a,b)))

[[1 1 1 1]
 [2 2 2 2]]
[[3 3]
 [4 4]]
[[1 1 1 1 3 3]
 [2 2 2 2 4 4]]
